In [ ]:
import alpaca_trade_api as alpaca
from dotenv import load_dotenv
import os

In [ ]:
# initialize API from API keys in .env
load_dotenv()
api_key = os.environ['APCA-API-KEY-ID']
api_secret_key = os.environ['APCA-API-SECRET-KEY']
api_base_url = 'https://paper-api.alpaca.markets'
api = alpaca.REST(api_key, api_secret_key, api_base_url)

In [ ]:
account = api.get_account()
print(account)

In [ ]:
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce

In [ ]:
trading_client = TradingClient(api_key, api_secret_key, paper=True)

In [ ]:
# test
# market_order_data = MarketOrderRequest(
#                     symbol="VOO",
#                     qty=0.5,
#                     side=OrderSide.BUY,
#                     time_in_force=TimeInForce.DAY
#                     )

# market_order = trading_client.submit_order(order_data=market_order_data)

In [ ]:
import pandas as pd
import talib as ta
import matplotlib.pyplot as plt

In [ ]:

print(len(ta.get_functions()))

In [ ]:
bars = api.get_bars("SPY", alpaca.TimeFrame.Day, "2021-06-01", "2024-11-18").df

bars


In [ ]:
bars['30_Day_SMA'] = ta.SMA(bars['close'], timeperiod=30)

plt.plot(bars['30_Day_SMA'], label='30 SMA')
plt.plot(bars['close'], label = 'close')
plt.legend()
plt.show()


In [ ]:
date = '2024-11-15'
market_open_datetime = date + "T14:30:00Z"
market_close_datetime = date + "T21:00:00Z"
bars2 = api.get_bars('SPY', alpaca.TimeFrame.Minute, market_open_datetime, market_close_datetime).df 

print(bars2.to_string())
bars2

In [ ]:
api.get_latest_quote('SPY')

In [ ]:
tickers = pd.read_csv('trading_tickers.csv')

tickers

In [ ]:
date = '2024-11-18'
market_at_1PM = date + "T18:00:00Z"

In [ ]:
import datetime as dt
now = dt.datetime.now(dt.timezone.utc)
todays_date = now.strftime("%Y-%m-%d")
market_open_datetime = todays_date + "T14:30:00Z"
current_datetime = now.strftime("%Y-%m-%dT%H:%M:%S.%fZ")
if now.hour >= 21:
    current_datetime = date + "T20:59:00Z"

print(market_open_datetime)
print(current_datetime)

In [ ]:
today_SPY_bars = api.get_bars("SPY", alpaca.TimeFrame(1, alpaca.TimeFrameUnit.Minute), market_open_datetime, feed='iex').df

today_SPY_bars

In [ ]:
# POTENTIAL STRATEGY
# here's an example of actual support levels
#   MSFT hits recent low of $400 on May 1st, then hits a similar low of $390 on Aug 5th
# however, good stocks tend to trend up. we can measure this with a 200 SMA
# we can then calculate adjusted support levels based on the difference from the 200 SMA, which acts as the expected trajectory
# we calculate the 200 SMA for the last 200 days, which requires 400 days of data

In [ ]:
import time
import datetime as dt

In [ ]:
def get_SP500():
    return pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

In [ ]:
SP500 = get_SP500()
SP500

In [ ]:
now = dt.datetime.now(dt.timezone.utc)
todays_date = now.strftime("%Y-%m-%d")
four_hundred_days_ago = (now - dt.timedelta(days=400)).strftime("%Y-%m-%d")
print(todays_date)
print(four_hundred_days_ago)
fh_date_time = four_hundred_days_ago + "T14:30:00Z"

bars = api.get_bars("SPY", alpaca.TimeFrame.Day, fh_date_time, feed='iex').df
bars